# Rapport sur les PVVIH de MUSO

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from dateutil.parser import parse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.by import By
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
from dotenv import load_dotenv
import plotly.express as px
import openpyxl
import pymysql
from sqlalchemy import create_engine,text


## Connect to commcare

In [3]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('COMCARE_EMAIL')
password_cc = os.getenv('COMCARE_PASSWORD')

#Defining the driver
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

svc = webdriver.ChromeService(executable_path=binary_path)
driver = webdriver.Chrome(service=svc)
driver.implicitly_wait(1000)

#Creating login function
def muso_beneficiaries():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce97cc9d15/'
    )
    #driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="id_auth-username"]').send_keys(email)
    #driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.XPATH,'//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR,'button[type=submit]').click()

#Muso beneficiaries
muso_beneficiaries()

#Download the database "All gardens"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,"//*[@id='download-export-form']/form/div[2]/div/div[2]/div[1]/button/span[1]").click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()    
driver.find_element(By.XPATH,"//*[@id='download-progress']/div/div/div[2]/div[1]/form/a/span[1]").click()    

#-----------------------------------------------------------------------------------------------------------------------

#Household count
def muso_groups():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce973b4c0d/'
    )
    
muso_groups()

#Download the database "Household count"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()


True

In [5]:
from time import sleep

sleep(50)

## Import the data

In [7]:
muso_benef = pd.read_excel(f"~/Downloads/muso_beneficiaries {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Cases')
muso_group = pd.read_excel(f"~/Downloads/muso_groupes {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")

In [8]:
muso_group.rename(columns = {'is_graduated': 'graduated_group',
                            'is_inactive': 'inactive_group',
                            'closed': 'closed_group',
                            'caseid': 'caseid_group',
                            'office': 'office_group'}, inplace = True)

muso_benef.rename(columns = {'indices.muso_groupes': 'caseid_group'}, inplace = True)

group_info = muso_group[['caseid_group', 'office_name', 'office_group', 'graduated_group', 'inactive_group', 'closed_group']]


In [9]:
muso_group.columns
muso_group.office_name.fillna("---",inplace=True)


Index(['number', 'caseid_group', 'name', 'gps', 'gps_date',
       'cycle_1_start_date', 'graduated_group', 'office_group',
       'graduation_date', 'localite_name', 'cycle_2_start_date', 'commune',
       'officer', 'cycle_3_end_date', 'commune_name', 'code',
       'cycle_2_end_date', 'cycle_1_end_date', 'inactive_date',
       'creation_date', 'meeting_hour', 'officer_name', 'meeting_day',
       'formed_by_members_from', 'localite', 'actual_cycle',
       'cycle_3_start_date', 'inactive_group', 'closed_group',
       'closed_by_username', 'closed_date', 'last_modified_by_user_username',
       'last_modified_date', 'opened_by_username', 'opened_date', 'owner_name',
       'case_link', 'case_type', 'closed_by_user_id', 'external_id',
       'last_modified_by_user_id', 'opened_by_user_id', 'owner_id',
       'server_last_modified_date', 'state', 'office_name', 'owner_id1',
       'section', 'section_name', 'departement', 'departement_name', 'adress',
       'office_location_id', 'co

In [10]:
actif_muso_group = muso_group[
                          (muso_group['graduated_group'] != '1') &
                         (muso_group['inactive_group'] != '1') & 
                         (muso_group['closed_group'] != True)]
actif_muso_group['office_name'] = np.where(actif_muso_group['office_name'] == '---', actif_muso_group['office_group'], actif_muso_group['office_name'])
actif_muso_group.shape
actif_muso_group.to_excel("./muso_groupes_actifs.xlsx",index=False,na_rep="---")

(584, 58)

## Concat to check good codes and to add info status to infos beneficiaries

In [11]:
muso_sheet = muso_benef.merge(group_info, on = 'caseid_group', how = 'left')
muso_sheet.shape


(56568, 81)

In [12]:
#connecting to HIV Haiti to get all codes on HIV Haiti
load_dotenv('.env')
user = os.getenv('USRCaris')
password_hiv = os.getenv('PASSCaris')
host = os.getenv('HOSTCaris')
db = os.getenv('DBCaris')

conn_text = f'mysql+pymysql://{user}:{password_hiv}@{host}/{db}'
engine = create_engine(conn_text)

sql2 = 'select * from patient'
all_patient = pd.read_sql_query(text(sql2), engine.connect(), parse_dates = True)
all_patient.rename(columns = {'patient_code': 'patient_code_pv'}, inplace = True)


False

In [13]:
all_patient['check_patient_code'] = all_patient['patient_code_pv']

all_patient.columns


Index(['id', 'city_code', 'hospital_code', 'patient_number', 'patient_code_pv',
       'linked_to_id_patient', 'which_program', 'muso_case_id',
       'schooling_positif_case_id', 'gardening_case_id',
       'schooling_ptme_case_id', 'dreams_case_id', 'created_at', 'updated_at',
       'created_by', 'updated_by', 'check_patient_code'],
      dtype='object')

In [14]:
# hhm_club = hhm_club.merge(OVC[['patient_code', 'Commune']], on = 'patient_code', how = 'left')

muso_sheet = muso_sheet.merge(all_patient[['patient_code_pv', 'check_patient_code']], on = 'patient_code_pv', how = 'left')

#merging is_pvvih and is_pvhiv
muso_sheet['is_pvvih'].fillna("---", inplace = True)
muso_sheet['is_pvvih'] = muso_sheet['is_pvhiv'].where(muso_sheet['is_pvvih'].isin(["---"]), 
                                           other=muso_sheet['is_pvvih'])


In [15]:
muso_sheet = muso_sheet[muso_sheet.group_name != 'test Shisu']


## Active benef and PVVIH

In [16]:
#filtering

active_benef = muso_sheet[(muso_sheet['why_remove_the_member'] == '---') & 
                          (muso_sheet['graduated_group'] != '1') &
                         (muso_sheet['inactive_group'] != '1') & 
                         (muso_sheet['closed_group'] != True)]

pvvih = active_benef[(active_benef['is_pvvih'] == '1')]


In [17]:
#muso_sheet.to_excel('Muso_data.xlsx', index = False)


In [18]:
actBen = active_benef
actBen['office_name'] = np.where(actBen['office_name'] == '---', actBen['office_group'], actBen['office_name'])


In [19]:
actBen.to_excel("./muso_active_members_on_active_groups.xlsx")


## pvvih must be caris member/

In [20]:
pvvih.is_caris_member.fillna("---",inplace=True)


In [21]:


def handle_isCarisMember(d):
    if d == "1":
        return "PTME"
    elif d == "2":
        return "OVC"
    elif d == "3":
        return "SCHOLARISATION"
    elif d == "4":
        return "DREAMS"
    elif d == "5":
        return "JARDINAGE"
    elif d == "6":
        return "PSYCHOSOCIAL"
    elif d == "7":
        return "FCI"
    elif d == "8":
        return "Aucun_Programme"
    else:
        return "---"



In [22]:
pvvih.is_caris_member = pvvih.is_caris_member.map(handle_isCarisMember)


In [23]:
pvvih.is_caris_member.value_counts()


PTME               742
---                549
Aucun_Programme    254
FCI                236
PSYCHOSOCIAL        52
SCHOLARISATION      42
OVC                 23
JARDINAGE           22
DREAMS               4
Name: is_caris_member, dtype: int64

## Infos global

In [24]:
print(f"Informations clés: Membres actifs : {active_benef.shape[0]} , PVVIH: {pvvih.shape[0]}.")


Informations clés: Membres actifs : 12523 , PVVIH: 1924.


In [26]:
active_benef.to_excel("./actif_benficiairies_muso.xlsx",index=False)

## Patient PV avec code

In [27]:
patient_code = pvvih[~(pvvih['patient_code_pv'].isin(['---', '//']))]
patient_code.shape[0]


1924

In [28]:
patient_code.to_excel(f"MUSO_PV_w_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", index = False)


## PV sans code



In [29]:
pv = pvvih[pvvih['patient_code_pv'].isin(['---', '//'])]
pv_wc = pv[['caseid', 'first_name', 'last_name', 'dob', 'gender', 'office_name', 'office_group','is_pvvih', 'patient_code_pv',
            'check_patient_code', 'group_name','group_code', 'group_commune', 'group_departement', 'phone', 'inactive_date', 'is_abandoned',
                   'officer_name', 'muso_start_date',"is_caris_member"]]

pv_wc['office_name'] = np.where(pv_wc['office_name'] == '---', pv_wc['office_group'], pv_wc['office_name'])
pv_wc.drop(['office_group'], axis = 1, inplace = True)
pv_wc['gender']=pd.to_numeric(pv_wc['gender'],errors="coerce")
pv_wc['gender']=pv_wc['gender'].replace([1,2],["M","F"])
pv_wc.head()


,caseid,first_name,last_name,dob,gender,office_name,is_pvvih,patient_code_pv,check_patient_code,group_name,group_code,group_commune,group_departement,phone,inactive_date,is_abandoned,officer_name,muso_start_date,is_caris_member


In [30]:
pv_wc.to_excel(f"MUSO_PVVIH_sans_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", index = False)


In [31]:
pv_wc.shape[0]


0

## Export final file

In [32]:
pv_wc.pivot_table(
    values="caseid",
    index="office_name",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
).to_excel("resume_pvvih_sans_code.xlsx",index=True)


In [33]:
pv_wc.columns


Index(['caseid', 'first_name', 'last_name', 'dob', 'gender', 'office_name',
       'is_pvvih', 'patient_code_pv', 'check_patient_code', 'group_name',
       'group_code', 'group_commune', 'group_departement', 'phone',
       'inactive_date', 'is_abandoned', 'officer_name', 'muso_start_date',
       'is_caris_member'],
      dtype='object')

In [34]:
pv_wc

,caseid,first_name,last_name,dob,gender,office_name,is_pvvih,patient_code_pv,check_patient_code,group_name,group_code,group_commune,group_departement,phone,inactive_date,is_abandoned,officer_name,muso_start_date,is_caris_member


In [35]:
if not pv_wc.empty:
    df = pv_wc
    column_name = 'office_name'
    unique_values = df[column_name].unique()

    with pd.ExcelWriter(f"./By_office/PV_sans_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", engine = 'openpyxl') as writer:
        for unique_value in unique_values:
            df_output = df[df[column_name].str.contains(unique_value)]
            df_output.to_excel(writer, sheet_name = unique_value, index = False)


In [36]:
patient_code = patient_code[['caseid', 'is_caris_member','first_name', 'last_name', 'dob', 'gender', 'office_name', 'office_group','is_pvvih', 'patient_code_pv',
            'check_patient_code', 'group_name','group_code', 'group_commune', 'group_departement', 'phone', 'inactive_date', 'is_abandoned',
                   'officer_name', 'muso_start_date']]
patient_code['office_name'] = np.where(patient_code['office_name'] == '---', patient_code['office_group'], patient_code['office_name'])
patient_code.drop(['office_group'], axis = 1, inplace = True)
patient_code['gender']=pd.to_numeric(patient_code['gender'],errors="coerce")
patient_code['gender']=patient_code['gender'].replace([1,2],["M","F"])

patient_code.check_patient_code.fillna("---",inplace=True)
df1 = patient_code
df1 = df1[df1.check_patient_code=="---"]

In [37]:
df1.shape
df1.is_caris_member.value_counts()

(354, 19)

Aucun_Programme    163
---                101
PTME                32
JARDINAGE           17
PSYCHOSOCIAL        12
FCI                 10
SCHOLARISATION      10
OVC                  7
DREAMS               2
Name: is_caris_member, dtype: int64

In [38]:
#pvCodeChecking_isCarisMemberMissing =df1[df1.is_caris_member=="---"]
#pvCodeChecking_isCarisMemberMissing.gender.value_counts()
#pvCodeChecking_isCarisMemberMissing.shape

pvCodeChecking_withIsCarisMember = df1[df1.is_caris_member!="---"]
pvCodeChecking_withIsCarisMember = pvCodeChecking_withIsCarisMember[pvCodeChecking_withIsCarisMember.is_caris_member!="Aucun_Programme"]
pvCodeChecking_withIsCarisMember.gender.value_counts()
pvCodeChecking_withIsCarisMember.shape


F    75
M    14
Name: gender, dtype: int64

(90, 19)

In [39]:
#column_name = 'office_name'
#unique_values = pvCodeChecking_isCarisMemberMissing[column_name].unique()

#with pd.ExcelWriter(f"./By_office/PV_checking_code_carisMemberMissing {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", engine = 'openpyxl') as writer:
#    for unique_value in unique_values:
#        pvCodeChecking_isCarisMemberMissing_output = pvCodeChecking_isCarisMemberMissing[pvCodeChecking_isCarisMemberMissing[column_name].str.contains(unique_value)]
#        pvCodeChecking_isCarisMemberMissing_output.to_excel(writer, sheet_name = unique_value, index = False)


In [40]:
column_name = 'office_name'
unique_values = pvCodeChecking_withIsCarisMember[column_name].unique()

with pd.ExcelWriter(f"./By_office/PV_checking_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", engine = 'openpyxl') as writer:
    for unique_value in unique_values:
        pvCodeChecking_withIsCarisMember_output = pvCodeChecking_withIsCarisMember[pvCodeChecking_withIsCarisMember[column_name].str.contains(unique_value)]
        pvCodeChecking_withIsCarisMember_output.to_excel(writer, sheet_name = unique_value, index = False)


In [41]:
pvCodeChecking_withIsCarisMember.pivot_table(
    values="caseid",
    index="office_name",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
).to_excel("resume_pvvih_code_check.xlsx",index=True)


In [42]:
#pvCodeChecking_isCarisMemberMissing.pivot_table(
#    values="caseid",
#    index="office_name",
#    aggfunc='count',
#    fill_value=0,
#    margins=True,
#    margins_name="Total"
#).to_excel("resume_pvvih_code_check_isCarisMemberMissing.xlsx",index=True)


In [43]:
driver.quit()
